### Limma voom analysis of Age Dataset

In [1]:
rm(list=ls())
#load necessary libraries 
library(ggplot2)
library("BiocParallel")
parallelFlag=TRUE
register(MulticoreParam(50))
library(sva)
library(limma)
library(statmod)

Loading required package: mgcv
Loading required package: nlme
This is mgcv 1.8-28. For overview type 'help("mgcv-package")'.
Loading required package: genefilter


## Load data and design

In [2]:
##load ATAC-seq raw read counts
data=read.table('../form_deseq_matrix/age.atac.counts.txt',header=TRUE,sep='\t')

##concatenate chrom/start/end columns values to server as rownames for the dataframe of the form chrom_start_end 
rownames(data)=paste(data$chrom,data$start,data$end,sep="_")
data$chrom=NULL
data$start=NULL
data$end=NULL

data=data[rowSums(data)>0,]


In [3]:
head(data)

,d0_Aged_Rep1,d0_Aged_Rep2,d0_Aged_Rep3,d0_Aged_Rep4,d0_Aged_Rep5,d0_Aged_Rep6,d0_Aged_Rep7,d0_Young_Pax7_Rep1,d0_Young_Pax7_Rep2,d0_Young_Pax7_Rep3,⋯,d5_Young_Rep2,d7_Aged_Rep1,d7_Aged_Rep2,d7_Aged_Rep3,d7_Aged_Rep4,d7_Young_Rep1,d7_Young_Rep2,d7_Young_Rep3,d7_Young_Rep4,d7_Young_Rep5
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
chr1_3060420_3060620,6,0,5,2,2,2,1,0,0,3,⋯,9,5,9,1,5,5,4,4,0,1
chr1_3100938_3101138,4,5,9,6,0,3,9,6,0,5,⋯,9,3,2,0,3,5,5,4,0,0
chr1_3103699_3103899,4,0,6,2,1,1,0,5,1,3,⋯,3,3,4,4,1,7,10,4,0,0
chr1_3119588_3119939,3,0,14,8,2,8,3,10,0,21,⋯,10,2,2,12,7,19,14,5,3,3
chr1_3119941_3120162,3,0,1,9,0,2,6,4,0,32,⋯,10,1,0,17,5,8,8,6,2,3
chr1_3212807_3213180,6,10,8,8,22,30,23,4,4,10,⋯,10,12,7,8,14,6,6,4,5,9


In [4]:
#load the metadata
batches=read.table("../batches.txt",header=TRUE,sep='\t')
batches$Batch=factor(batches$Batch)
batches$Pax7=factor(batches$Pax7)
batches$Age=factor(batches$Age)


In [5]:
cpm=voom(data,normalize.method = "quantile")
E=cpm$E
E=round(E,2)

## Correlation 

In [6]:
spearman_cor=cor(E, method = "spearman")

In [7]:
pearson_cor=cor(E,method="pearson")

In [8]:
write.table(spearman_cor,file="atac.uncorrected.spearman_r.tsv",quote=FALSE)
write.table(pearson_cor,file="atac.uncorrected.pearson_r.tsv",quote=FALSE)

In [9]:
## plot correlation heatmaps 
library(gplots)
require(gtools)
require(RColorBrewer)
cols <- colorRampPalette(brewer.pal(10, "RdBu"))(256)

svg(filename="ATAC_rep_spearman_cor_preSVA.svg",
   height=8,
   width=8,
   pointsize=12)
cur_h=heatmap.2(as.matrix(spearman_cor), 
          trace="none", 
          scale="none", 
          Rowv=TRUE,
          Colv=TRUE,
          col=rev(cols), 
          dendrogram="none",
          margins=c(10,10),
          main="ATAC-seq replicates spearman correlation -- pre-SVA")
dev.off() 



Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess

Loading required package: gtools

Attaching package: ‘gtools’

The following object is masked from ‘package:mgcv’:

    scat

Loading required package: RColorBrewer


png 
  2

In [10]:
dev.off() 

null device 
          1

In [11]:

svg(filename="ATAC_rep_pearson_cor_preSVA.svg",
   height=8,
   width=8,
   pointsize=12)
cur_h=heatmap.2(as.matrix(pearson_cor), 
          trace="none", 
          scale="none", 
          Rowv=TRUE,
          Colv=TRUE,
          col=rev(cols), 
          dendrogram="none",
          margins=c(10,10),          
          main="ATAC-seq replicates pearson correlation -- pre-SVA")

dev.off() 

png 
  2

## Run PCA 

In [12]:
data.pca=prcomp(t(E))

In [13]:
var_explained=as.character(round(100*data.pca$sdev^2/sum(data.pca$sdev^2),2))

In [14]:
svg(filename="age_atac_uncorrected_scree.svg",
   height=3,
   width=4,
   pointsize=12)
print(barplot(100*data.pca$sdev^2/sum(data.pca$sdev^2),las=2,ylab="% Variance Explained",xlab="Principal Component",ylim=c(0,40), xlim=c(0,10)))
dev.off() 

      [,1]
 [1,]  0.7
 [2,]  1.9
 [3,]  3.1
 [4,]  4.3
 [5,]  5.5
 [6,]  6.7
 [7,]  7.9
 [8,]  9.1
 [9,] 10.3
[10,] 11.5
[11,] 12.7
[12,] 13.9
[13,] 15.1
[14,] 16.3
[15,] 17.5
[16,] 18.7
[17,] 19.9
[18,] 21.1
[19,] 22.3
[20,] 23.5
[21,] 24.7
[22,] 25.9
[23,] 27.1
[24,] 28.3
[25,] 29.5
[26,] 30.7
[27,] 31.9
[28,] 33.1
[29,] 34.3
[30,] 35.5
[31,] 36.7
[32,] 37.9
[33,] 39.1
[34,] 40.3
[35,] 41.5
[36,] 42.7
[37,] 43.9
[38,] 45.1
[39,] 46.3
[40,] 47.5


png 
  2

In [15]:
pca_df=data.frame(data.pca$x)
pca_df=cbind(pca_df,batches)
pca_df$Day=factor(pca_df$Day)

In [16]:
svg(filename="age_atac_uncorrected_PC1_vs_PC2_batch.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC1,y=pca_df$PC2,color=pca_df$Batch,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#000000'))+
theme_bw() )
dev.off() 

png 
  2

In [17]:
svg(filename="age_atac_uncorrected_PC2_vs_PC3_batch.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC2,y=pca_df$PC3,color=pca_df$Batch,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#000000'))+
theme_bw())
dev.off() 

png 
  2

In [18]:
svg(filename="age_atac_uncorrected_PC1_vs_PC3_batch.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC1,y=pca_df$PC3,color=pca_df$Batch,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#000000'))+
theme_bw())
dev.off() 

png 
  2

## Running SVA correction 

In [19]:
head(batches)

Rep,Day,Age,Pax7,Batch
<fct>,<int>,<fct>,<fct>,<fct>
d0_Aged_Rep1,0,Aged,0,b2074
d0_Aged_Rep2,0,Aged,0,b2074
d0_Aged_Rep3,0,Aged,0,b2236
d0_Aged_Rep4,0,Aged,0,b2322
d0_Aged_Rep5,0,Aged,0,b2863
d0_Aged_Rep6,0,Aged,0,b2863


In [20]:
Grouping <- factor(paste0(batches$Day,".",batches$Age, ".", batches$Pax7))
batches$Grouping=Grouping

In [21]:
mod0=model.matrix(~1,data=batches)
#mod1=model.matrix(~Day+Age+Pax7+Batch,data=batches)
mod1=model.matrix(~0+Grouping+Batch,data=batches)

In [22]:
sva.obj=sva(E,mod1,mod0)

Number of significant surrogate variables is:  9 
Iteration (out of 5 ):1  2  3  4  5  

In [23]:
sur_var=data.frame(sva.obj$sv)

In [24]:
full.design.sv=cbind(mod1,sur_var)


In [25]:
#save the full design so we don't have to run SVA next time 
write.table(full.design.sv,file="full.design.sv.txt",quote=FALSE,sep='\t')


In [26]:
#full.design.sv=read.table("full.design.sv.txt",header=TRUE,sep='\t')


### Perform pca on the "cleaned" matrix with surrogate variable contributions removed


In [27]:
nrow(batches)

[1] 40

In [28]:
cleaned_E=removeBatchEffect(E,batch=batches$Batch,covariates=sur_var,design=model.matrix(~0+Grouping,data=batches))


In [29]:
## Run replicate correlation on SVA corrected dataset 
spearman_cor=cor(cleaned_E, method = "spearman")
pearson_cor=cor(cleaned_E,method="pearson")

In [30]:
write.table(spearman_cor,file="atac.corrected.spearman_r.tsv",quote=FALSE)
write.table(pearson_cor,file="atac.corrected.pearson_r.tsv",quote=FALSE)

In [31]:

svg(filename="ATAC_rep_spearman_cor_postSVA.svg",
   height=8,
   width=8,
   pointsize=12)
cur_h=heatmap.2(as.matrix(spearman_cor), 
          trace="none", 
          scale="none", 
          Rowv=TRUE,
          Colv=TRUE,
          col=rev(cols), 
          dendrogram="none",
          margins=c(12,12),          
          main="ATAC-seq replicates spearman correlation -- post-SVA ")
dev.off() 



png 
  2

In [32]:
svg(filename="ATAC_rep_pearson_cor_postSVA.svg",
   height=8,
   width=8,
   pointsize=12)

cur_h=heatmap.2(as.matrix(pearson_cor), 
          trace="none", 
          scale="none", 
          Rowv=TRUE,
          Colv=TRUE,
          col=rev(cols), 
          dendrogram="none",
          margins=c(12,12),          
          main="ATAC-seq replicates pearson correlation -- post SVA")
dev.off() 


png 
  2

In [33]:
data.pca=prcomp(t(cleaned_E))

In [34]:
var_explained=as.character(round(100*data.pca$sdev^2/sum(data.pca$sdev^2),2))


In [35]:
svg(filename="age_atac_corrected_scree.svg",
   height=3,
   width=4,
   pointsize=12)
print(barplot(100*data.pca$sdev^2/sum(data.pca$sdev^2),las=2,ylab="% Variance Explained",xlab="Principal Component",ylim=c(0,40), xlim=c(0,10)))
dev.off()

      [,1]
 [1,]  0.7
 [2,]  1.9
 [3,]  3.1
 [4,]  4.3
 [5,]  5.5
 [6,]  6.7
 [7,]  7.9
 [8,]  9.1
 [9,] 10.3
[10,] 11.5
[11,] 12.7
[12,] 13.9
[13,] 15.1
[14,] 16.3
[15,] 17.5
[16,] 18.7
[17,] 19.9
[18,] 21.1
[19,] 22.3
[20,] 23.5
[21,] 24.7
[22,] 25.9
[23,] 27.1
[24,] 28.3
[25,] 29.5
[26,] 30.7
[27,] 31.9
[28,] 33.1
[29,] 34.3
[30,] 35.5
[31,] 36.7
[32,] 37.9
[33,] 39.1
[34,] 40.3
[35,] 41.5
[36,] 42.7
[37,] 43.9
[38,] 45.1
[39,] 46.3
[40,] 47.5


png 
  2

In [36]:
var_explained[0:10]


[1] "20.04" "10.21" "9.56"  "6.47"  "5.75"  "4.57"  "4.49"  "3.75"  "3.65" 
[10] "3.49"

In [37]:
pca_df=data.frame(data.pca$x)
pca_df=cbind(pca_df,batches)
pca_df$Day=factor(pca_df$Day)

In [38]:
svg(filename="age_atac_corrected_PC1_vs_PC2.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC1,y=pca_df$PC2,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00'))+
theme_bw())
dev.off()

png 
  2

In [39]:
svg(filename="age_atac_corrected_PC2_vs_PC3.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC2,y=pca_df$PC3,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00'))+
theme_bw())
dev.off()

png 
  2

In [40]:
svg(filename="age_atac_corrected_PC1_vs_PC3.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC1,y=pca_df$PC3,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00'))+
theme_bw())
dev.off()

png 
  2

### fit model with limma 

In [41]:
#fit <- lmFit(E,full.design.sv)
fit = lmFit(cleaned_E,model.matrix(~0+Grouping))

In [42]:
colnames(fit)


[1] "Grouping0.Aged.0"  "Grouping0.Young.0" "Grouping0.Young.1"
 [4] "Grouping1.Aged.0"  "Grouping1.Young.0" "Grouping3.Aged.0" 
 [7] "Grouping3.Young.0" "Grouping5.Aged.0"  "Grouping5.Young.0"
[10] "Grouping7.Aged.0"  "Grouping7.Young.0"

In [43]:
colnames(fit$coefficients)

[1] "Grouping0.Aged.0"  "Grouping0.Young.0" "Grouping0.Young.1"
 [4] "Grouping1.Aged.0"  "Grouping1.Young.0" "Grouping3.Aged.0" 
 [7] "Grouping3.Young.0" "Grouping5.Aged.0"  "Grouping5.Young.0"
[10] "Grouping7.Aged.0"  "Grouping7.Young.0"

###  Contrasts 

In [44]:
#create contrasts of interest 
cont.matrix=makeContrasts(
    d0_Y_vs_A="Grouping0.Young.0 - Grouping0.Aged.0",  
    d1_Y_vs_A="Grouping1.Young.0 - Grouping1.Aged.0",
    d3_Y_vs_A="Grouping3.Young.0 - Grouping3.Aged.0",
    d5_Y_vs_A="Grouping5.Young.0 - Grouping5.Aged.0",
    d7_Y_vs_A="Grouping7.Young.0 - Grouping7.Aged.0",
    d1_Y_vs_d0_Y="Grouping1.Young.0 - Grouping0.Young.0",
    d1_A_vs_d0_A="Grouping1.Aged.0 - Grouping0.Aged.0",
    d3_Y_vs_d1_Y="Grouping3.Young.0 - Grouping1.Young.0",
    d3_A_vs_d1_A="Grouping3.Aged.0 - Grouping1.Aged.0",
    d5_Y_vs_d3_Y="Grouping5.Young.0 - Grouping3.Young.0",
    d5_A_vs_d3_A="Grouping5.Aged.0 - Grouping3.Aged.0",
    d7_Y_vs_d5_Y="Grouping7.Young.0 - Grouping5.Young.0",
    d7_A_vs_d5_A="Grouping7.Aged.0 - Grouping5.Aged.0",
    d0_Y_Pax_7_vs_d0_Y_noPax7="Grouping0.Young.1 - Grouping0.Young.0",
    levels=model.matrix(~0+Grouping))


In [45]:
pval_thresh=0.01
lfc_thresh=1

In [46]:
fit2=contrasts.fit(fit,cont.matrix)
e=eBayes(fit2)
comparisons=colnames(cont.matrix)

In [47]:
for(i in seq(1,length(comparisons)))
{
  tab<-topTable(e, number=nrow(e),coef=i,lfc=lfc_thresh, p.value = pval_thresh)
  up=sum(tab$logFC>0)
  down=sum(tab$logFC<0)
  sig=nrow(tab)
  curtitle=paste(comparisons[i],'\n','sig:',sig,'\n','up:',up,'\n','down:',down,'\n')
  print(curtitle)
  vals=topTable(e,number=nrow(e),coef=i)
  vals$pscaled=-1*log10(vals$adj.P.Val)
  vals$sig=vals$adj.P.Val<pval_thresh & abs(vals$logFC)>lfc_thresh 
  png(paste("volcano_diff",comparisons[i],".png",sep=""))
  print(ggplot(data=vals,
               aes(y=vals$pscaled,x=vals$logFC,color=vals$sig))+
               geom_point(alpha=0.1)+
               xlab("log2(FC)")+
               ylab("-log10(pval)")+
               ggtitle(curtitle)+
               theme_bw()+
               scale_color_manual(values=c("#000000","#FF0000")))
  dev.off() 
  write.table(tab,file=paste("diff_",comparisons[i],".tsv",sep=""),quote=FALSE,sep='\t',row.names = TRUE,col.names = TRUE)
}
    

[1] "d0_Y_vs_A \n sig: 43046 \n up: 21194 \n down: 21852 \n"
[1] "d1_Y_vs_A \n sig: 21764 \n up: 11889 \n down: 9875 \n"
[1] "d3_Y_vs_A \n sig: 282 \n up: 179 \n down: 103 \n"
[1] "d5_Y_vs_A \n sig: 11586 \n up: 5739 \n down: 5847 \n"
[1] "d7_Y_vs_A \n sig: 24813 \n up: 12855 \n down: 11958 \n"
[1] "d1_Y_vs_d0_Y \n sig: 39301 \n up: 19395 \n down: 19906 \n"
[1] "d1_A_vs_d0_A \n sig: 58867 \n up: 29334 \n down: 29533 \n"
[1] "d3_Y_vs_d1_Y \n sig: 69055 \n up: 33591 \n down: 35464 \n"
[1] "d3_A_vs_d1_A \n sig: 62782 \n up: 31468 \n down: 31314 \n"
[1] "d5_Y_vs_d3_Y \n sig: 20879 \n up: 9642 \n down: 11237 \n"
[1] "d5_A_vs_d3_A \n sig: 20933 \n up: 9817 \n down: 11116 \n"
[1] "d7_Y_vs_d5_Y \n sig: 32316 \n up: 17601 \n down: 14715 \n"
[1] "d7_A_vs_d5_A \n sig: 22273 \n up: 11915 \n down: 10358 \n"
[1] "d0_Y_Pax_7_vs_d0_Y_noPax7 \n sig: 34141 \n up: 17617 \n down: 16524 \n"


In [49]:
topTable(e)

,d0_Y_vs_A,d1_Y_vs_A,d3_Y_vs_A,d5_Y_vs_A,d7_Y_vs_A,d1_Y_vs_d0_Y,d1_A_vs_d0_A,d3_Y_vs_d1_Y,d3_A_vs_d1_A,d5_Y_vs_d3_Y,d5_A_vs_d3_A,d7_Y_vs_d5_Y,d7_A_vs_d5_A,d0_Y_Pax_7_vs_d0_Y_noPax7,AveExpr,F,P.Value,adj.P.Val
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr10_104244318_104244518,-2.966033,-1.08637623,-0.01448627,-0.59053222,-5.303721,-6.658213,-8.537870,8.965596,7.893706,-7.930016,-7.353970,2.2588058,6.971995,-2.489510,2.577579,528.4427,1.125475e-33,2.684967e-28
chr8_4656715_4656923,-1.999319,1.20950559,0.61820592,1.13895557,-4.032018,-3.260727,-6.469552,5.394890,5.986190,-5.394242,-5.914992,0.8969099,6.067883,-2.504218,1.402779,277.8377,4.467335e-29,3.809489e-24
chr17_22876400_22876602,-1.794323,2.07787138,0.52712987,-0.43473278,-4.797786,-3.082190,-6.954385,5.284909,6.835651,-7.960901,-6.999038,2.8856956,7.248749,-3.046847,2.870165,276.6574,4.790545e-29,3.809489e-24
chr1_142729889_142730090,-3.273219,-0.82055502,0.36916070,-0.81077059,-6.079459,-5.325273,-7.777937,8.556542,7.366827,-9.426556,-8.246625,2.8316363,8.100325,-2.337698,2.523585,267.0445,8.556626e-29,5.103236e-24
chr1_24613316_24613516,-1.632953,0.61904722,-0.06819192,0.29730265,-4.294446,-3.730984,-5.982984,4.343100,5.030339,-5.544739,-5.910234,1.6207364,6.212485,-1.633654,6.912836,256.0101,1.708930e-28,6.231091e-24
chr11_3159822_3160045,-2.534243,0.84872329,-0.08015588,-0.48217843,-5.061516,-2.487189,-5.870155,3.910627,4.839506,-6.878539,-6.476517,2.6496802,7.229018,-1.944942,1.806340,254.5949,1.871434e-28,6.231091e-24
chr1_188978272_188978478,-2.639239,0.26640569,0.11884807,-0.07666495,-5.110811,-3.559781,-6.465426,6.300422,6.447980,-6.848585,-6.653072,1.1085877,6.142734,-1.180450,4.002649,254.5872,1.872366e-28,6.231091e-24
chr1_24612248_24612448,-1.529424,0.47409110,-0.07771836,0.24413979,-4.636751,-3.863247,-5.866763,4.543033,5.094842,-5.938494,-6.260352,1.7996452,6.680536,-2.218153,6.531863,252.8877,2.089541e-28,6.231091e-24
chr1_171070288_171070800,-1.068217,0.05833574,0.23468395,0.62933918,-4.656344,-5.093353,-6.219905,5.482666,5.306318,-6.151279,-6.545934,1.7964078,7.082091,-3.363986,1.442507,249.9677,2.527450e-28,6.699513e-24


In [50]:
nrow(cleaned_E)

[1] 238563

In [51]:
cleaned_E_df=as.data.frame(cleaned_E)
head(cleaned_E_df)

,d0_Aged_Rep1,d0_Aged_Rep2,d0_Aged_Rep3,d0_Aged_Rep4,d0_Aged_Rep5,d0_Aged_Rep6,d0_Aged_Rep7,d0_Young_Pax7_Rep1,d0_Young_Pax7_Rep2,d0_Young_Pax7_Rep3,⋯,d5_Young_Rep2,d7_Aged_Rep1,d7_Aged_Rep2,d7_Aged_Rep3,d7_Aged_Rep4,d7_Young_Rep1,d7_Young_Rep2,d7_Young_Rep3,d7_Young_Rep4,d7_Young_Rep5
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1_3060420_3060620,0.2980111,-0.4949106,-0.8352578,-0.1160573,0.2730918,-1.2475962,-0.8879229,-1.90072660,-0.08176152,-0.8222272,⋯,-0.05084371,-1.464294,-0.11006068,-1.3047446,0.1712703,-1.2683797,-1.82190825,-1.899839012,-2.0046108,-1.65164836
chr1_3100938_3101138,0.1849240,1.5523861,1.5793781,1.2666634,-0.2395032,0.6429433,2.2516198,1.43304206,-0.58963195,0.2204195,⋯,-0.32854850,-2.141257,-2.26703297,-1.3599426,-0.5035253,-1.1222927,-2.07613874,-1.834772792,-1.8082717,-2.40409078
chr1_3103699_3103899,-1.8132443,-1.3379290,-1.0882839,-1.2455845,-0.2638067,-1.8023063,-1.9987796,-0.09346071,-1.52124420,-0.9930103,⋯,-1.62595165,-1.147750,0.07087044,-0.5156485,-0.9869808,-0.2602057,-0.08722435,-0.009158339,0.2355524,0.04249603
chr1_3119588_3119939,-1.3741031,0.2536048,1.1944576,-0.1375741,0.1435114,0.2128912,-0.1758270,1.02677433,-1.42451178,0.1306106,⋯,0.67643144,-2.216750,-1.40418049,-0.7770812,-0.0192496,1.8052421,1.24270281,1.457381996,1.4233681,0.79838427
chr1_3119941_3120162,-0.7314200,-0.6006032,-0.2880665,-0.6325003,-1.6280749,-1.5048052,0.5280731,0.10359787,-1.23938086,-0.2683879,⋯,0.49333440,-1.613924,-1.99574760,-1.0122742,-0.3800374,1.2092776,0.52187464,0.740642792,0.6748637,0.24646564
chr1_3212807_3213180,1.2863337,2.6202389,2.4499429,2.5089691,2.5984299,2.1845759,2.1230662,0.57735879,2.02451064,0.4180125,⋯,0.38018376,0.654511,-0.30475782,-0.0590653,0.3272756,-0.1682086,-0.57601229,-0.719008485,-0.2828867,-0.87231518


## Plot d0_Y vs d0_Y_Pax7

In [53]:
d0_Y=(cleaned_E_df$d0_Young_Rep1+
      cleaned_E_df$d0_Young_Rep2+
      cleaned_E_df$d0_Young_Rep3+
      cleaned_E_df$d0_Young_Rep4)/4
d0_Y_Pax7=(cleaned_E_df$d0_Young_Pax7_Rep1+
          cleaned_E_df$d0_Young_Pax7_Rep2+
          cleaned_E_df$d0_Young_Pax7_Rep3)/3

In [54]:
d0_Y_df=data.frame(rownames(cleaned_E_df),d0_Y,d0_Y_Pax7)

In [55]:
head(d0_Y_df)

rownames.cleaned_E_df.,d0_Y,d0_Y_Pax7
<fct>,<dbl>,<dbl>
chr1_3060420_3060620,-0.3046422,-0.93490512
chr1_3100938_3101138,0.5820489,0.35460987
chr1_3103699_3103899,-0.7047459,-0.86923840
chr1_3119588_3119939,1.0688200,-0.08904229
chr1_3119941_3120162,-0.7906023,-0.46805697
chr1_3212807_3213180,0.1724557,1.00662732


In [57]:
svg(filename="ATAC_d0Y_vs_d0YPax7.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=d0_Y_df,
      aes(x=d0_Y_df$d0_Y,
         y=d0_Y_df$d0_Y_Pax7))+
geom_point(alpha=0.3)+
geom_abline(slope=1)+
xlab("D0 Young")+
ylab("D0 Young PAX7")+
ggtitle("cpm ATAC-seq counts")+
theme_bw(20))
dev.off()

png 
  2